In [ ]:
import requests

from FPL import FPL

import warnings
warnings.filterwarnings("ignore")

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()
df = FPL(json).getPlayerDf()

# Aggregate

In [16]:
# Avg value per position
FPL(json).createPivot('element_type', 'value_season').head()

,element_type,value_season
2,Goalkeeper,15.064000
0,Defender,12.521101
3,Midfielder,10.474667
1,Forward,9.219048


In [17]:
# Avg value per team
FPL(json).createPivot('team', 'value_season').head()

,team,value_season
11,Man City,15.142857
10,Liverpool,13.711765
18,West Ham,13.693333
0,Arsenal,13.162500
2,Brentford,12.900000


In [18]:
# Avg form per team
FPL(json).createPivot('team', 'form').head()

,team,form
10,Liverpool,4.535294
11,Man City,4.000000
16,Spurs,3.643750
5,Chelsea,3.589474
6,Crystal Palace,3.580000


In [19]:
# Avg value form per team
FPL(json).createPivot('team', 'value_form').head()

,team,value_form
6,Crystal Palace,0.693333
10,Liverpool,0.647059
1,Aston Villa,0.637500
16,Spurs,0.625000
5,Chelsea,0.610526


# Top 10 players overall

In [ ]:
# Sorting by value season
df = FPL(json).getPlayerDf()
df.sort_values('value_season', ascending=False).head(10)


In [ ]:
# Sorting by form
df = FPL(json).getPlayerDf()
df.sort_values('form', ascending=False).head(10)


In [ ]:
# Sorting by value form
df = FPL(json).getPlayerDf()
df.sort_values('value_form', ascending=False).head(10)


# Top per position

In [12]:
position = 'Forward'

In [13]:
# Value
FPL(json).dfPerPosition(position, 'value_season').head()

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,value_form
626,Dennis,Watford,Forward,60,2174.0,19.2,115,2.0,0.3
499,Pukki,Norwich,Forward,58,2445.0,17.9,104,5.2,0.9
84,Toney,Brentford,Forward,67,2188.0,16.3,109,8.0,1.2
651,Antonio,West Ham,Forward,75,2486.0,15.2,114,2.0,0.3
552,Broja,Southampton,Forward,55,1510.0,14.7,81,1.8,0.3


In [14]:
# Form
FPL(json).dfPerPosition(position, 'form').head()

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,value_form
566,Kane,Spurs,Forward,125,2421.0,10.9,136,8.8,0.7
84,Toney,Brentford,Forward,67,2188.0,16.3,109,8.0,1.2
457,dos Santos Aveiro,Man Utd,Forward,122,1914.0,9.6,117,6.3,0.5
499,Pukki,Norwich,Forward,58,2445.0,17.9,104,5.2,0.9
66,Ings,Aston Villa,Forward,77,1520.0,10.8,83,5.0,0.6


In [15]:
# Value form
FPL(json).dfPerPosition(position, 'value_form').head()

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,value_form
84,Toney,Brentford,Forward,67,2188.0,16.3,109,8.0,1.2
631,Hernández Suárez,Watford,Forward,50,1005.0,11.8,59,4.8,1.0
234,Mateta,Crystal Palace,Forward,52,647.0,7.3,38,4.8,0.9
499,Pukki,Norwich,Forward,58,2445.0,17.9,104,5.2,0.9
566,Kane,Spurs,Forward,125,2421.0,10.9,136,8.8,0.7


## Per aggregate